In [1]:
# train_model.py
import pandas as pd
import numpy as np
import joblib
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import os

# Dosya İsimleri
DATA_FILE = "train.csv"
MODEL_FILE = "superstore_models.pkl"

def train_and_save():
    print("⏳ Veri seti yükleniyor ve işleniyor...")
    
    if not os.path.exists(DATA_FILE):
        print(f"❌ HATA: '{DATA_FILE}' bulunamadı!")
        return

    # Veriyi Oku
    df = pd.read_csv(DATA_FILE)
    
    # Tarih formatını düzelt (Gün/Ay/Yıl -> Datetime)
    df['Order Date'] = pd.to_datetime(df['Order Date'], format='%d/%m/%Y')
    
    # Veriyi Aylık ve Kategori Bazlı Grupla
    # Her ayın toplam satışını hesaplıyoruz
    df_monthly = df.groupby(['Category', pd.Grouper(key='Order Date', freq='ME')])['Sales'].sum().reset_index()
    
    categories = df_monthly['Category'].unique()
    models = {}
    
    print("🧠 Yapay zeka modelleri eğitiliyor...")
    
    for cat in categories:
        print(f"   ➡️ Kategori işleniyor: {cat}")
        
        # Sadece ilgili kategorinin verisini al
        cat_data = df_monthly[df_monthly['Category'] == cat].set_index('Order Date')['Sales']
        
        # Modeli Eğit (Holt-Winters: Trend + Mevsimsellik)
        try:
            # Full model (Trend + Mevsimsellik)
            model = ExponentialSmoothing(
                cat_data,
                seasonal_periods=12,
                trend='add',
                seasonal='add'
            ).fit()
        except:
            # Veri azsa veya hata verirse sadece Trend kullan
            print(f"      ⚠️ Uyarı: {cat} için tam mevsimsellik yakalanamadı, basit trend kullanılıyor.")
            model = ExponentialSmoothing(
                cat_data,
                trend='add'
            ).fit()
            
        models[cat] = model

    # Modelleri Kaydet
    joblib.dump(models, MODEL_FILE)
    print(f"\n✅ BAŞARILI! Tüm modeller '{MODEL_FILE}' dosyasına kaydedildi.")

if __name__ == "__main__":
    train_and_save()

⏳ Veri seti yükleniyor ve işleniyor...
🧠 Yapay zeka modelleri eğitiliyor...
   ➡️ Kategori işleniyor: Furniture
   ➡️ Kategori işleniyor: Office Supplies
   ➡️ Kategori işleniyor: Technology

✅ BAŞARILI! Tüm modeller 'superstore_models.pkl' dosyasına kaydedildi.


C:\Users\wfi-7\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
C:\Users\wfi-7\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\wfi-7\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
C:\Users\wfi-7\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\wfi-7\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency ME will be used.
  self._init_dates(dates, freq)
C:\Users\wfi-7\anaconda3\Lib\site-